In [31]:
import yaml
from yaml.loader import SafeLoader
from socket import gethostname
import numpy as np
import pandas as pd
from sklearn.base import clone
from dev_interaction_util import generate_synthetic_dev_outcomes, generate_synthetic_dev_data, set_up_interactions
from dev_interaction_util import do_scoring_loop, get_best_model, summarize_overall_df_results, do_final_fit, present_model_results, present_results_vs_ground_truth_cors
from dev_interaction_util import load_and_preprocess_data, impute_data, run_full_limited_predictor_analysis
from ml_util import *
# Imputing with MICE
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn import linear_model
from ml_util import get_data_for_imputation
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.inspection import permutation_importance
import numpy as np
from IPython.display import display, HTML
from sklearn.base import clone
from sklearn.inspection import permutation_importance
import seaborn as sns
from sklearn.feature_selection import SelectKBest, f_regression, RFE


In [32]:


print(gethostname())
# Open the file and load the file
with open('config.yml') as f:
    all_yaml = yaml.load(f, Loader=SafeLoader)
    if gethostname() in all_yaml.keys():
        config = all_yaml[gethostname()]
    else:
        config = all_yaml['default']
        
print(config)



Benjamins-MacBook-Pro-2.local
{'dropbox_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/'}


This notebook is derived from `test_feature_selection.ipynb`.

In [33]:
dropbox_data_dir = config['dropbox_data_dir']


In [34]:
analysis_data, outcome_measures = load_and_preprocess_data(dropbox_data_dir)

In [35]:
analysis_data_imputed = impute_data(analysis_data)

/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis3_10/lib/python3.10/site-packages/sklearn/impute/_iterative.py:713: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [36]:


def do_hyperparameter_selection_loop_r2(X,y,cv):
    return(do_hyperparameter_selection_loop_w_metric(X,y,cv,'r2'))

#loops through the different estimators and feature selection methods and does a grid search over all to find the best hyperparameters
def do_hyperparameter_selection_loop(X, y,cv):
    return(do_hyperparameter_selection_loop_w_metric(X,y,cv,'neg_mean_absolute_error'))

#loops through the different estimators and feature selection methods and does a grid search over all to find the best hyperparameters
def do_hyperparameter_selection_loop_w_metric(X, y,cv,metric):
    #alpha parameters for Ridge and Lasso
    alpha_10pow_lower = 1
    alpha_10pow_upper = 0
    alpha_increments=1
    alpha_range = np.concatenate([np.power(10,np.linspace(-alpha_10pow_lower,alpha_10pow_upper,(alpha_10pow_lower+alpha_10pow_upper)*alpha_increments+1)),
        [0.2,0.3,0.4,0.6,0.8,1.0]])
    
    all_cv_results = []

    pipeline_estimator_name = 'estimator'
    feature_selection_name = 'feature_selection'


    #define the param_grid for the estimators
    estimators_to_run = {
        'Ridge':{
            'estimator':linear_model.Ridge,
            'parameters':{'alpha':alpha_range}
        },
        'Lasso':{
            'estimator':linear_model.Lasso,
            'parameters':{'alpha':alpha_range}
        },
        'DecisionTreeRegressor':{
            'estimator':DecisionTreeRegressor,
            'parameters':{
                'max_depth':[2, 4],
                'min_samples_split':[20,50],
                'min_samples_leaf':[20,50]
            }
        }             
    }

    k_max_val = np.min([50,X.shape[1]])

    for estimator_name,estimator_dict in estimators_to_run.items():
        #param grid for the feature seelction
        #this is here because we need to know the estimator to pass to the feature selector
        feature_selectors_to_run = {
            'None':None,
            'KBest':{
                'selector':SelectKBest(),
                'parameters':{
                    'score_func' : [f_regression], 
                    'k' : [10,25,k_max_val]
                    }
            },
            'RFE':{
                'selector':RFE(linear_model.LinearRegression()),
                'parameters':{
                    'n_features_to_select' : [10,25],
                    #'verbose':[1],
                    'step':[5]
                }
            }
        }
        for selector_name, selector_dict in feature_selectors_to_run.items():
        #create the estimator
            if selector_name == 'None':
                pipeline = Pipeline([('scaler',StandardScaler()),
                                     (pipeline_estimator_name,estimator_dict['estimator']())])
                selector_params = {}
            else:
                pipeline = Pipeline([('scaler',StandardScaler()),
                                     (feature_selection_name,selector_dict['selector']), 
                                     (pipeline_estimator_name,estimator_dict['estimator']())])
                selector_params = selector_dict['parameters']

            estimator_param_grid = {(pipeline_estimator_name + '__'+k):v for k,v in estimator_dict['parameters'].items()}
            selector_param_grid = {(feature_selection_name + '__'+k):v for k,v in selector_params.items()}
            #combine the two param grid dictionaries
            full_param_grid = {**selector_param_grid, **estimator_param_grid}
            print(pipeline)
            print(full_param_grid)

            
        
            gs_1 = GridSearchCV(estimator=pipeline, 
                                param_grid = full_param_grid, 
                                cv=cv,scoring=metric,verbose=1)
            gs_1.fit(X,y)
            all_cv_results.append(gs_1)

    #create a dataframe with the best parameters, best mean_test_score, and name of the model

    best_params_df = pd.DataFrame({
        'model': [cv_result.estimator for cv_result in all_cv_results],
        'model_name': [cv_result.estimator.__class__.__name__ for cv_result in all_cv_results],
        'best_params': [extract_estimator_params_from_gridsearch(cv_result.best_params_) for cv_result in all_cv_results],
        'best_score': [cv_result.best_score_ for cv_result in all_cv_results],
        'best_raw_params' : [cv_result.best_params_ for cv_result in all_cv_results]
        })
    
    best_params_df = best_params_df.sort_values('best_score',ascending=False).reset_index(drop=True)

    best_model = clone(best_params_df['model'][0])
    best_model_params = best_params_df['best_raw_params'][0]
    best_model.set_params(**best_model_params)

    return {
        'best_model': best_model,
        'best_params_df':best_params_df,
        'raw_cv_results':all_cv_results
    }


# Improving fit with manual theory-driven feature

My past analysis showed that by manually removing some features before the analysis starts, we can improve performance beyond the chance performance otherwise seen.

So, it might be useful to understand how much we can improve our performance by manual feature selection before the automatic feature selection applies.

This was previously done in `test_limited_predictors.ipynb`. We tested as few as 2 distractor features. In that test, predictor features generally had correlations in the range of |r|=0.06 to 0.53, with most around 0.4 (we should confirm that because it seems fishy that PCS was detegted as an effect, but didn't model as a large predictor). With most `|r|=0.4`, this seems unrealistically high to expect, and we should aim to build a pipeline capable of detecting more subtle effects than that. An approximate `|r|=0.3` can be achieved by mixing in a predictor scaled to 8% of normal scale.

I can imagine it is plausible to cut down to as few as two self-report, one behavioral, and one neural measure per intervention, plus sex and age. That would yield 10 different variables. At the other end, we might want 10 self-report, two behavioral, and five neural measures per intervention tested, plus 6 different demographic variables--a total of 40 variables. Let's see how these would perform, as well as mid-range of 20 predictor variables. In each case we'll restrict to three valid predictors per intervention.

In [37]:
overall_scores = pd.DataFrame(columns=['n_features','effect_size', 'overall_score'])
        
        
hypers = {
    'r2':do_hyperparameter_selection_loop_r2,
    'mae':do_hyperparameter_selection_loop
}
for pcount in [10,15]:
    for effect_size in [0.07,0.08,0.10]:
        for ho in ['r2','mae']:
            overall_score = run_full_limited_predictor_analysis(
                pcount,
                outcome_measures,
                analysis_data_imputed,
                effect_size= effect_size,
                hyperparameter_optimizer = hypers[ho]
                )

            #run the analysis with a limited number of predictors
            overall_scores = overall_scores.append(
                {'n_features':pcount,
                'effect_size':effect_size,
                'overall_score':overall_score,
                'hyper_target':ho},
                ignore_index=True)

['ni' 'san']
[1.28335298 0.42953651]
['san' 'san' 'ni' 'ichi' 'san' 'san' 'ichi' 'san' 'san' 'san' 'ni' 'ichi'
 'ichi' 'ichi' 'ichi' 'san' 'san' 'san' 'ichi' 'ichi' 'san' 'san' 'ni'
 'ni' 'ni' 'ni' 'ni' 'ni' 'ni' 'san' 'ni' 'san' 'ni' 'ichi' 'ni' 'san'
 'ni' 'ichi' 'san' 'ni' 'ni' 'ichi' 'ichi' 'ichi' 'san' 'san' 'ni' 'ni'
 'san' 'ichi' 'ichi' 'ni' 'san' 'ni' 'ichi' 'ni' 'ni' 'ni' 'ichi' 'san'
 'ni' 'ni' 'ichi' 'ni' 'ichi' 'san' 'ni' 'ni' 'ni' 'san' 'ichi' 'ni' 'san'
 'ichi' 'san' 'ni' 'san' 'ni' 'ichi' 'ichi' 'san' 'ichi' 'san' 'san' 'ni'
 'ichi' 'ni' 'ichi' 'san' 'ni' 'san' 'ni' 'ichi' 'san' 'san' 'san' 'ichi'
 'ni' 'san' 'ichi' 'ichi' 'san' 'ni' 'ichi' 'san' 'ni' 'ni' 'san' 'ni'
 'ichi' 'ni' 'ichi' 'ichi' 'ni' 'ichi' 'ichi' 'ichi' 'san' 'san' 'ichi'
 'ni' 'ni' 'ichi' 'ni' 'ni' 'ichi' 'ichi' 'san' 'san' 'ni' 'ichi' 'ni'
 'ichi' 'ichi' 'san' 'ichi' 'ni' 'san' 'san' 'ni' 'ni' 'san' 'san' 'san'
 'ichi' 'san' 'ni' 'san' 'ichi' 'ichi' 'ichi' 'ni' 'san' 'ni' 'ni' 'ni'
 'ichi' 'ni' 'ichi' '

doing permutation test on importance; this may take time.
Number of selected features: 3


,predictor,coef,feature_importance,fa_abs
12,BSCS*ni,0.680126,0.072934,0.072934
26,RS*san,0.200386,0.011315,0.011315
6,ACES_neglectful_parenting,-0.130812,0.003058,0.003058
10,ni,0.000000,0.000000,0.000000
11,san,0.000000,0.000000,0.000000
14,BIS_11*ni,-0.000000,0.000000,0.000000
17,TRSQ*ni,0.000000,0.000000,0.000000
18,ACES_neglectful_parenting*ni,0.000000,0.000000,0.000000
20,ACES_sum*ni,-0.000000,0.000000,0.000000
27,TRSQ*san,0.000000,0.000000,0.000000


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:358: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  results_vs_cors = final_results_wide.merge(group_correlations, left_index=True, right_index=True, how='outer')


,"(coef, base)","(coef, ni)","(coef, san)","(feature_importance, base)","(feature_importance, ni)","(feature_importance, san)",ichi_cor,ni_cor,san_cor,abs_effect_sum
BSCS,NaN,0.68,NaN,NaN,0.073,NaN,-0.137,0.314,0.017,0.073
RS,NaN,NaN,0.2,NaN,NaN,0.011,0.039,-0.141,0.236,0.011
ACES_neglectful_parenting,-0.131,0.00,NaN,0.003,0.000,NaN,-0.046,-0.019,-0.194,0.003


...

doing permutation test on importance; this may take time.
Number of selected features: 19


,predictor,coef,feature_importance,fa_abs
17,BSCS*ni,1.520835,0.280856,0.280856
36,RS*san,0.601340,0.053674,0.053674
18,EDM*ni,0.521283,0.037669,0.037669
38,ACES_neglectful_parenting*san,-0.454181,0.030541,0.030541
2,BIS_11,-0.443326,0.030219,0.030219
19,BIS_11*ni,-0.452073,0.028686,0.028686
31,BFI_neuroticism*ni,-0.448212,0.028388,0.028388
13,BFI_extraversion,0.331615,0.022058,0.022058
45,BFI_extraversion*san,0.301765,0.017198,0.017198
11,BFI_agreeableness,-0.325563,0.014493,0.014493


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:358: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  results_vs_cors = final_results_wide.merge(group_correlations, left_index=True, right_index=True, how='outer')


,"(coef, base)","(coef, ni)","(coef, san)","(feature_importance, base)","(feature_importance, ni)","(feature_importance, san)",ichi_cor,ni_cor,san_cor,abs_effect_sum
BSCS,-0.000,1.521,-0.000,0.000,0.281,0.000,-0.137,0.415,-0.011,0.281
BIS_11,-0.443,-0.452,-0.000,0.030,0.029,0.000,0.047,-0.440,-0.033,0.059
RS,0.108,-0.000,0.601,0.004,0.000,0.054,0.039,-0.177,0.304,0.058
EDM,0.119,0.521,-0.000,0.003,0.038,0.000,0.053,0.287,-0.065,0.040
BFI_extraversion,0.332,0.000,0.302,0.022,0.000,0.017,NaN,NaN,NaN,0.039
ACES_neglectful_parenting,-0.072,0.000,-0.454,0.002,0.000,0.031,-0.046,-0.014,-0.262,0.032
BFI_neuroticism,-0.000,-0.448,0.000,0.000,0.028,0.000,NaN,NaN,NaN,0.028
BFI_agreeableness,-0.326,0.000,-0.000,0.014,0.000,0.000,NaN,NaN,NaN,0.014
ACES_divorced_separated,-0.000,-0.000,-0.263,0.000,0.000,0.010,NaN,NaN,NaN,0.010
ACES_household_dysfunction,0.037,-0.190,-0.000,0.000,0.006,0.000,NaN,NaN,NaN,0.006


/var/folders/cj/4mb6t1f906j397tj71pxfxz00000gn/T/ipykernel_56060/3502319901.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  overall_scores = overall_scores.append(


In [39]:
overall_scores

,n_features,effect_size,overall_score,hyper_target
0,10,0.07,-0.048384,r2
1,10,0.07,-0.046887,mae
2,10,0.08,-0.089049,r2
3,10,0.08,-0.083980,mae
4,10,0.10,-0.035533,r2
5,10,0.10,-0.048563,mae
6,15,0.07,-0.045498,r2
7,15,0.07,-0.104665,mae
8,15,0.08,-0.046213,r2
9,15,0.08,-0.078301,mae
